In [0]:
import pandas as pd
import numpy as np
import datetime

from pyspark import SparkConf, SparkContext
from pyspark.sql import SQLContext
from pyspark.sql.types import *
from pyspark import SparkConf, SparkContext
from pyspark.sql import Row, SparkSession

# Output Schema

In [0]:
### OUTPUT SCHEMA ###
"""
meta_id
model_created
model_run
model_name
model_ver
user
data_ver
sell_in_out
periodicity
comments
sector
category
brand
sub_brand
product_name
sales_org
sold_to
sold_to_description
distribution_channel
country
active
score
id
week
month
year
ean
actual_quantity
price_per_case
promo_price
base_price
Net_Invoice_LC
Net_Invoice_USD
lower_limit
upper_limit
accuracy_10
accuracy_50
accuracy_90
predicted_quantity
baseline_quantity
fk_ml_meta
"""

Out[2]: '\nmeta_id\nmodel_created\nmodel_run\nmodel_name\nmodel_ver\nuser\ndata_ver\nsell_in_out\nperiodicity\ncomments\nsector\ncategory\nbrand\nsub_brand\nproduct_name\nsales_org\nsold_to\nsold_to_description\ndistribution_channel\ncountry\nactive\nscore\nid\nweek\nmonth\nyear\nean\nactual_quantity\nprice_per_case\npromo_price\nbase_price\nNet_Invoice_LC\nNet_Invoice_USD\nlower_limit\nupper_limit\naccuracy_10\naccuracy_50\naccuracy_90\npredicted_quantity\nbaseline_quantity\nfk_ml_meta\n'

# Helper Functions

In [0]:
def format_datetime(full_df):
  
  formatted_data = full_df.copy()
  formatted_data = full_df.groupby(['SOLD_TO', 'EAN', 'YEARWEEK']).sum().reset_index()
  formatted_data.rename(columns = {'YEARWEEK': 'WEEK'}, inplace = True)
  formatted_data['YEARWEEK'] = [datetime.datetime.strptime(d + '-1', "%Y%W-%w") for d in formatted_data['WEEK']]
  
  return formatted_data

# Read in the data

## Model predictions and baseline

In [0]:
## READ IN PREDICTIONS
pred_actuals = spark.sql('select * from default.peru_data_pred_actuals_all').toPandas()

In [0]:
## READ IN BASELINE
bl_actuals = spark.sql('select * from default.peru_data_bl_actuals_all').toPandas()

In [0]:
bl_actuals[['CUSTOMER', 'EAN']] = bl_actuals[['CUSTOMER', 'EAN']].astype(str)
pred_actuals[['CUSTOMER', 'EAN']] = pred_actuals[['CUSTOMER', 'EAN']].astype(str)

In [0]:
bl_actuals = bl_actuals[bl_actuals['CAL_WEEK'] >= datetime.datetime(2021, 1, 1)]
pred_actuals = pred_actuals[pred_actuals['CAL_WEEK'] >= datetime.datetime(2021, 1, 1)]

## Original Data

### EAN Descriptions

In [0]:
# Connection object to Snowflake
dev_url = "https://devkcc.east-us-2.azure.snowflakecomputing.com"
qa_url = "https://qakcc.east-us-2.azure.snowflakecomputing.com"
dev_user = "APP_ADF_PROMOANALYTICS_DEV"
qa_user = "APP_ADF_PROMOANALYTICS_QA"
dev_password = "VSG6eC0bqjSZxkr8"
qa_password = "IgwOidxm4tTmP7dY"
sfOptions = { 
  "sfUrl" : dev_url, 
  "sfUser" : dev_user, 
  "sfPassword" : dev_password, 
  "sfDatabase" : "RGM_ADV_ANLTCS", 
  "sfSchema" : "INT", 
  "sfWarehouse" : "RGM_SALES_GBL_BI_WH"
} 

In [0]:
query = "SELECT DISTINCT A.EAN11 AS EAN, (A.EAN11||'-'||A.PRODUCT_DESC) AS EAN_DESCRIPTION, A.PRODUCT_CD, A.PRODUCT_DESC FROM RGM_SALES_GBL.INT_MD.RGM_LAO_MD_DIM_PRODUCT A INNER JOIN (SELECT DISTINCT EAN11, LAST_VALUE(PRODUCT_CD) OVER (PARTITION BY EAN11 ORDER BY PRODUCT_CD) AS PRODUCT FROM RGM_SALES_GBL.INT_MD.RGM_LAO_MD_DIM_PRODUCT WHERE SALES_ORG = '7140' AND EAN11 !=' ' AND BASE_UOM = 'CS' ORDER BY 1) B ON A.EAN11=B.EAN11 AND A.PRODUCT_CD=B.PRODUCT ORDER BY 1,2"

In [0]:
ean_desc_df = spark.read.format("snowflake").options(**sfOptions).option("query", query).load().toPandas()
ean_desc_df['EAN'] = ean_desc_df['EAN'].astype(str)

In [0]:
query = "SELECT * FROM RGM_ADV_ANLTCS.STAGING.V_RGM_LAO_PE_PO_NIS_ADV_NEW1" 

In [0]:
full_df = spark.read.format("snowflake").options(**sfOptions).option("query", query).load().toPandas()
full_df = full_df.drop_duplicates()

/databricks/spark/python/pyspark/sql/pandas/utils.py:79: UserWarning: The conversion of DecimalType columns is inefficient and may take a long time. Column names: [YEAR, MONTH, WEEK, PROMO_DURATION] If those columns are not necessary, you may consider dropping them or converting to primitive types before the conversion.
 warnings.warn(

In [0]:
full_df.rename(columns = {'WEEK': 'YEARWEEK'}, inplace = True)
full_df['SOLD_TO'] = full_df['SOLD_TO'].astype(int).astype(str)
full_df['YEARWEEK'] = full_df['YEARWEEK'].astype(str)
full_df['SALES_VOLUME'] = full_df['SALES_VOLUME'].astype(float)
full_df['SALES_VALUE'] = full_df['SALES_VALUE'].astype(float)
full_df['NETINV_USD'] = full_df['SALES_VALUE'] * 0.25
full_df['EAN'] = full_df['EAN'].astype(str)

In [0]:
full_df.drop(['PROMO_INVESTMENT', 'PROMO_DURATION'], axis = 1, inplace = True)

In [0]:
full_df = full_df.rename(columns = {'PRODUCT_DESC': 'PRODUCT_NAME'})

In [0]:
full_df['EAN_DESCRIPTION'] = full_df['EAN'] + '-' + full_df['PRODUCT_NAME']
full_df['EAN_DESCRIPTION'].isnull().any()

Out[16]: False

In [0]:
full_df['WEEK'] = full_df['YEARWEEK'].str.slice(start = -2)

### Aggregated Numerical Values

In [0]:
agg_num_full_df = format_datetime(full_df)

In [0]:
joined_ean_desc = full_df.copy()

## Hyperparameter table

In [0]:
output_files = ['peru_all_data_500', 
               'peru_all_data_500_1000_csv']

hp_table = pd.DataFrame()
for file in output_files:
    df = spark.sql('select * from default.' + file).toPandas()
    df.dropna(inplace = True)
    df = df[df['val_mape'] != float('inf')]
    df[['Customer', 'OSKU']] = df[['Customer', 'OSKU']].astype(str)
    df[['lag', 'ma']] = df[['lag', 'ma']].astype(int)
    df['end_date'] = pd.to_datetime(df['end_date'])
    df['trend'] = df['trend'].str.strip('[]').str.split(', ')
    hp_table = pd.concat([hp_table, df])
    
hp_table.drop('_c0', axis = 1, inplace = True)
hp_table['start_date'] = 0

In [0]:
hp_table[['Customer', 'OSKU']] = hp_table[['Customer', 'OSKU']].astype(str)

In [0]:
hp_table = hp_table[~hp_table[['Customer', 'OSKU']].duplicated()]
hp_table = hp_table[['Customer', 'OSKU', 'val_mape']].copy()

# Join tables according to the Output Schema

## Join baseline and topline predictions

In [0]:
joined_bl_pred_actuals = bl_actuals.merge(pred_actuals, how = 'left', on = ['CUSTOMER', 'EAN', 'CAL_WEEK'])
joined_bl_pred_actuals.drop('actual_x', axis = 1, inplace = True)

## Join Customer and OSKU hierarchies

In [0]:
desc_cols = ['SOLD_TO', 'EAN', 'SECTOR', 'CATEGORY', 'SUB_CATEGORY', 'BRAND', 'SUB_BRAND', 'DISTRIBUTION_CHANNEL', 'CUSTOMER_NAME', 'EAN_DESCRIPTION', 'PLAN_LEVEL', 'PRODUCT_NAME']
cust_osku_desc = joined_ean_desc[desc_cols].drop_duplicates(subset = ['SOLD_TO', 'EAN'])
joined_cust_osku_desc = joined_bl_pred_actuals.merge(cust_osku_desc, how = 'left', left_on = ['CUSTOMER', 'EAN'], right_on = ['SOLD_TO', 'EAN'])
joined_cust_osku_desc.drop('SOLD_TO', axis = 1, inplace = True)

In [0]:
# weeks_master_df
# Cols: Date - actual full date (for example, 2022-01-27)
#       week - 202204
start_date = 202101
end_date = 202253
weeks_master_list = [str(yearweek) for yearweek in range(start_date, end_date + 1) if yearweek % 100 <= 53]
weeks_master_df = pd.DataFrame(weeks_master_list, columns = ['WEEK'])
weeks_master_df['DATE'] = [datetime.datetime.strptime(d + '-1', "%Y%W-%w") for d in weeks_master_df['WEEK']]
weeks_master_df

Out[25]:

WEEK 
 DATE 
 
 
 
 
 0 
 202101 
 2021-01-04 
 
 
 1 
 202102 
 2021-01-11 
 
 
 2 
 202103 
 2021-01-18 
 
 
 3 
 202104 
 2021-01-25 
 
 
 4 
 202105 
 2021-02-01 
 
 
 ... 
 ... 
 ... 
 
 
 102 
 202249 
 2022-12-05 
 
 
 103 
 202250 
 2022-12-12 
 
 
 104 
 202251 
 2022-12-19 
 
 
 105 
 202252 
 2022-12-26 
 
 
 106 
 202253 
 2023-01-02 
 
 
 
 107 rows × 2 columns

In [0]:
joined_cust_osku_desc = joined_cust_osku_desc.merge(weeks_master_df, how = 'left', left_on = 'CAL_WEEK', right_on = 'DATE')

In [0]:
test = joined_cust_osku_desc.merge(agg_num_full_df[['SOLD_TO', 'EAN', 'YEARWEEK', 'WEEK', 'SALES_VOLUME', 'SALES_VALUE', 'NETINV_USD']], 
                                  how = 'left', left_on = ['CUSTOMER', 'EAN', 'WEEK'], right_on = ['SOLD_TO', 'EAN', 'WEEK'])
test.update(test[['SALES_VOLUME','SALES_VALUE','NETINV_USD']].fillna(0))
test.loc[test['SALES_VOLUME'] < 0, 'SALES_VOLUME'] = 0
test.loc[test['actual_y'] == -1, 'SALES_VOLUME'] = -1
test.drop(['SOLD_TO', 'YEARWEEK', 'CAL_WEEK'], axis = 1, inplace = True)
test.rename(columns = {'WEEK': 'CAL_WEEK'}, inplace = True)

In [0]:
dupes_test = test['CUSTOMER'] + '-' + test['EAN'] + '-' + test['CAL_WEEK']

In [0]:
joined_cust_osku_desc = test.copy()
joined_cust_osku_desc.drop('actual_y', axis = 1, inplace = True)
joined_cust_osku_desc.rename(columns = {'SALES_VOLUME': 'actual_y'}, inplace = True)

## Join Sales Value Columns

In [0]:
joined_sales_values = joined_cust_osku_desc.copy()

## Generate year, month and week columns

In [0]:
joined_sales_values['week'] = joined_sales_values['CAL_WEEK'].str.slice(start = -2).astype(int)
joined_sales_values['month'] = pd.DatetimeIndex(joined_sales_values['DATE']).month
joined_sales_values.loc[joined_sales_values['week'] == 53, 'month'] = 12
joined_sales_values['year'] = joined_sales_values['CAL_WEEK'].str.slice(stop = -2).astype(int)

In [0]:
joined_sales_values.drop('CAL_WEEK', axis = 1, inplace = True)

## Join MAPE values

In [0]:
joined_errors = joined_sales_values.merge(hp_table[['Customer', 'OSKU', 'val_mape']], how = 'left', right_on = ['Customer', 'OSKU'], left_on = ['CUSTOMER', 'EAN'])
joined_errors.drop(['Customer', 'OSKU'], axis = 1, inplace = True)

In [0]:
joined_errors.drop('EAN', axis = 1, inplace = True)

# Common fixes

## Add in 'common value' columns

In [0]:
timestamp = datetime.datetime.now()
joined_errors['model_created'] = timestamp
joined_errors['model_run'] = timestamp
joined_errors['model_name'] = 'peru_sell_in_out_model_r4_new_data'
joined_errors['model_ver'] = '9.0'
joined_errors['user'] = 'lokesh.valluri'
joined_errors['data_ver'] = '8.0'
joined_errors['sell_in_out'] = 'out'
joined_errors['periodicity'] = 'weekly'
joined_errors['comments'] = 'Test comment'
joined_errors['active'] = True
joined_errors['sales_org'] = '7140'
joined_errors['country'] = 'PE'

In [0]:
joined_errors['price_per_case'] = 0
joined_errors['promo_price'] = 0
joined_errors['base_price'] = 0
joined_errors['accuracy_10'] = 0
joined_errors['accuracy_50'] = 0
joined_errors['accuracy_90'] = 0

## Fill NA values with appropriate values

In [0]:
joined_errors['val_mape'].fillna(-1, inplace = True)

## Add in an ID column

In [0]:
joined_errors['id'] = joined_errors.index + 1

## Rename columns

In [0]:
joined_errors['day'] = 1

In [0]:
joined_errors.rename(columns = {'SALES_VALUE': 'Net_Invoice_LC', 
                               'NETINV_USD': 'Net_Invoice_USD', 
                               'low_ci': 'lower_limit', 
                               'high_ci': 'upper_limit', 
                               'SECTOR': 'sector', 
                               'CATEGORY': 'category', 
                               'SUB_CATEGORY': 'sub_category',
                               'BRAND': 'brand', 
                               'SUB_BRAND': 'sub_brand', 
                               'PRODUCT_NAME': 'product_name', 
                               'CUSTOMER': 'sold_to', 
                               'CUSTOMER_NAME': 'sold_to_description', 
                               'EAN_DESCRIPTION': 'ean', 
                               'DISTRIBUTION_CHANNEL': 'distribution_channel', 
                               'val_mape': 'score', 
                               'actual_y': 'actual_quantity', 
                               'baseline': 'baseline_total_quantity', 
                               'predicted': 'predicted_quantity', 
                               'PLAN_LEVEL': 'plan_level'}, 
                    inplace = True)

In [0]:
new_order_str = 'id    day    week    month    year    actual_quantity    price_per_case    promo_price    base_price    Net_Invoice_LC    Net_Invoice_USD    lower_limit    upper_limit    accuracy_10    accuracy_50    accuracy_90    predicted_quantity    baseline_total_quantity    ean    model_created    model_run    model_name    model_ver    user    data_ver    sell_in_out    periodicity    comments    sector    category    brand    sub_brand    product_name    sales_org    sold_to    sold_to_description    distribution_channel    country    active    score    sub_category    plan_level'
cols_new_order = [col_name for col_name in new_order_str.split('    ')]
joined_errors = joined_errors[cols_new_order]

## Replace commas with a period

In [0]:
str_cols = ['ean', 'comments', 'sector', 'category', 'brand', 'sub_brand', 'product_name', 'sold_to_description', 'sub_category']
joined_errors[str_cols] = joined_errors[str_cols].apply(lambda x: x.str.replace(',','.'))

In [0]:
joined_errors.loc[joined_errors['lower_limit'] == joined_errors['baseline_total_quantity'], 'lower_limit'] = 0

In [0]:
#start_of_split = joined_errors['product_name'].str.find('-')
joined_errors['product_name'] = [p_name[p_name.find('-') + 1:] for p_name in joined_errors['product_name']]

In [0]:
joined_errors = joined_errors.fillna(0)

In [0]:
spark_df = spark.createDataFrame(joined_errors)

# Write to Snowflake

In [0]:
# Execute the update / delete command
sfSourceName = "net.snowflake.spark.snowflake"
spark = SparkSession.builder.appName("snowflake").getOrCreate()
spark.sparkContext._jvm.net.snowflake.spark.snowflake.Utils.runQuery(sfOptions, "update RGM_ADV_ANLTCS.INT.RGM_SLS_FCT_DATA_BASELINE set ACTIVE = 0 where COUNTRY = 'PE';")
spark.stop

Out[47]: <bound method SparkSession.stop of <pyspark.sql.session.SparkSession object at 0x7f6315b37160>>

In [0]:
spark.createDataFrame(joined_errors).write.format("snowflake").mode("append").options(**sfOptions).option("dbtable", "RGM_ADV_ANLTCS.INT.RGM_SLS_FCT_DATA_BASELINE").save()

In [0]:
query = "select MODEL_RUN from RGM_ADV_ANLTCS.INT.RGM_SLS_FCT_DATA_BASELINE where COUNTRY = 'PE' and ACTIVE = 1"
spark_df = spark.read.format("snowflake").options(**sfOptions).option("query", query).load().toPandas()

In [0]:
timestamp = spark_df['MODEL_RUN'].unique()[0]
log_df = pd.DataFrame(columns = ['MODEL_RUN', 'DATE_CREATED', 'DATE_COMPLETED', 'IN_PROGRESS', 'COMPLETED', 'COUNTRY'])
log_df.loc[0] = [timestamp, timestamp, timestamp, False, False, 'PE']

In [0]:
spark.createDataFrame(log_df).write.format("snowflake").mode("append").options(**sfOptions).option("dbtable", "RGM_ADV_ANLTCS.INT.RGM_SLS_FCT_DATA_BASELINE_ADJUSTMENT_LOG").save()

In [0]:
s_df = spark.createDataFrame(log_df)

In [0]:
# Connection object to Snowflake
dev_url = "https://devkcc.east-us-2.azure.snowflakecomputing.com"
qa_url = "https://qakcc.east-us-2.azure.snowflakecomputing.com"
dev_user = "APP_ADF_PROMOANALYTICS_DEV"
qa_user = "APP_ADF_PROMOANALYTICS_QA"
dev_password = "VSG6eC0bqjSZxkr8"
qa_password = "IgwOidxm4tTmP7dY"
sfOptions = { 
  "sfUrl" : qa_url, 
  "sfUser" : qa_user, 
  "sfPassword" : qa_password, 
  "sfDatabase" : "RGM_ADV_ANLTCS", 
  "sfSchema" : "INT", 
  "sfWarehouse" : "RGM_SALES_GBL_BI_WH"
} 